# Neural Nets from Scratch in Julia

## Lesson : Tanh

* In this video we'll implement the tanh function for *Values*.
* [Documentation site here](https://mikesaint-antoine.github.io/SimpleGrad.jl)
* [Github repo here](https://github.com/mikesaint-antoine/SimpleGrad.jl)

In [1]:
## code so far

mutable struct Value{opType} <: Number
    data::Float64
    grad::Float64
    op::opType
end


struct Operation{FuncType, ArgTypes}
    op::FuncType
    args::ArgTypes
end

# constructor -- Value(data, grad, op)
Value(x::Number) = Value(Float64(x), 0.0, nothing);


import Base.show
function show(io::IO, value::Value)
    print(io, "Value(",value.data,")")
end


import Base.==
function ==(a::Value, b::Value)
     return a===b
end


import Base.+
function +(a::Value, b::Value)

    out = a.data + b.data    
    result = Value(out, 0.0, Operation(+, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



backprop!(val::Value{Nothing}) = nothing


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(+), ArgTypes}
    
    # val = a + b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.grad
    val.op.args[2].grad += val.grad
    
end




function backward(a::Value)
    
    
    function build_topo(v::Value, visited=Value[], topo=Value[])
    
        if !(v in visited)
            
            push!(visited, v)
            
            if v.op != nothing
                for operand in v.op.args
                    
                    if operand isa Value
                        build_topo(operand, visited, topo)
                    end
                end 
            end
            
            push!(topo, v) 
            
            
        end
        return topo
    end
    
    
    
    topo = build_topo(a)
    
    a.grad = 1
    #da/da = 1
    
    for node in reverse(topo)
        backprop!(node)
    end
    
    
end


Base.promote_rule(::Type{<:Value}, ::Type{T}) where {T<:Number} = Value




import Base.*
function *(a::Value, b::Value)

    out = a.data * b.data    
    result = Value(out, 0.0, Operation(*, (a,b))) # Value(data, grad, op)
    return result # this should be a Value
 
end



function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(*), ArgTypes}
    
    # val = a * b
    # update a.grad, b.grad
    
    val.op.args[1].grad += val.op.args[2].data * val.grad    
    val.op.args[2].grad += val.op.args[1].data * val.grad
    
end



import Base.-

# negation
function -(a::Value)
    
    return a * -1
    
end

# subtraction
function -(a::Value, b::Value)
    
    return a + (-b)
    
    
end


import Base.inv
function inv(a::Value)
    
    out = 1.0 / a.data
    result = Value(out, 0.0, Operation(inv, (a,))) # Value(data, grad, op)
    return result # this should be a Value    
    
    
end


function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(inv), ArgTypes}
    
    # val = inv(a)
    # update a.grad
    
    # a.grad -= (1.0 / a.data^2) * val.grad
    
    val.op.args[1].grad -= (1.0 / val.op.args[1].data^2) * val.grad
    
    
end


import Base./
function /(a::Value, b::Value)
     
    # a/b = a * b^(-1)
    
    return a * inv(b)
    
    
end

/ (generic function with 107 methods)

In [5]:
import Base.tanh
function tanh(a::Value)
    
    out = (exp(2 * a.data) - 1) / (exp(2 * a.data) + 1)
    result = Value(out, 0.0, Operation(tanh, (a,))) # Value(data, grad, op)
    return result # this should be a Value  
    
end




function backprop!(val::Value{Operation{FunType, ArgTypes}}) where {FunType<:typeof(tanh), ArgTypes}

    # val = tanh(a)
    # update a.grad
    
    val.op.args[1].grad += (1 - val.data^2) * val.grad
    

end





# tanh(input1*w1 + input2*w2 + input3*w3 + bias)


backprop! (generic function with 5 methods)

In [6]:
input1 = Value(2.3)
input2 = Value(-3.5)
input3 = Value(3.9)

weight1 = Value(-0.8)
weight2 = Value(1.8)
weight3 = Value(3.0)

bias = Value(-3.2)


neuron_output = tanh(input1*weight1 + input2*weight2 + input3*weight3 + bias)


backward(neuron_output)

println(weight1.grad)
println(weight2.grad)
println(weight3.grad)

println(bias.grad)





2.0259027224625243
-3.082895447225581
3.435226355479933
0.8808272706358803
